In [2]:
# Importar librerias
import pandas as pd
import json
from pandas import json_normalize
from textblob import TextBlob
import numpy as np

In [3]:
# Apertura y visualización del archivo australian_user_reviews
user_reviews = []
with open ('./file_json/australian_user_reviews.json', 'r', encoding='utf-8') as file:
    data = file.readlines()
registros = [eval(line.strip()) for line in data]
Df_reviews = pd.DataFrame(registros)
Df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [4]:
# La columna 'reviews' tiene datos anidados, por lo que es necesario desglosarla para generar nuevas filas para cada elemento contenido en ella.
Df_reviews = Df_reviews.explode('reviews', ignore_index=True)
Df_reviews_new = json_normalize(Df_reviews['reviews'])

In [5]:
# Se combinan los DataFrames
Df_reviews = Df_reviews.join(Df_reviews_new)
Df_reviews.head()

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [6]:
# Se eliminan las columnas que no seran utilizadas para el proceso de modelado
Df_reviews.drop(['user_url','reviews', 'funny', 'last_edited', 'helpful'], axis=1, inplace=True)

In [7]:
# Se identifican registros duplicados y los eliminamos
duplicados = Df_reviews.duplicated()
Df_reviews = Df_reviews.drop_duplicates()
# Se eliminan los registros nulos
Df_reviews = Df_reviews.dropna(how='all')

In [8]:
# Se extrae el año de la columna posted
# Me aseguro que 'posted' sea de tipo datetime
Df_reviews['posted'] = pd.to_datetime(Df_reviews['posted'].astype(str).str.replace(r'Posted |,|\.', '', regex=True), errors='coerce')
# Crea la columna 'year' a partir de 'posted'
Df_reviews['year'] = Df_reviews['posted'].dt.year.astype('Int64')
Df_reviews.head()

,user_id,posted,item_id,recommend,review,year
0,76561197970982479,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,2011-07-15,22200,True,It's unique and worth a playthrough.,2011
2,76561197970982479,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,2014-06-24,251610,True,I know what you think when you see this title ...,2014
4,js41637,2013-09-08,227300,True,For a simple (it's actually not all that simpl...,2013


In [9]:
#Se eliminan los nulos de year
Df_reviews.dropna(subset='year', inplace=True)
Df_reviews

,user_id,posted,item_id,recommend,review,year
0,76561197970982479,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,2011
1,76561197970982479,2011-07-15,22200,True,It's unique and worth a playthrough.,2011
2,76561197970982479,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011
3,js41637,2014-06-24,251610,True,I know what you think when you see this title ...,2014
4,js41637,2013-09-08,227300,True,For a simple (it's actually not all that simpl...,2013
...,...,...,...,...,...,...
59280,wayfeng,2015-10-14,730,True,its FUNNNNNNNN,2015
59283,76561198251004808,2015-10-10,253980,True,Awesome fantasy game if you don't mind the gra...,2015
59293,72947282842,2015-10-31,730,True,Prettyy Mad Game,2015
59295,ApxLGhost,2015-12-14,730,True,AMAZING GAME 10/10,2015


In [10]:
#Se crea una función para detectar el sentimiento
def sentimiento(texto):
        analisis = TextBlob(texto)
        #Se obtiene la polaridad
        polaridad = analisis.sentiment.polarity
        if polaridad < 0:
            return 0  # Negativo
        elif polaridad > 0:
            return 2  # Positivo
        else:
            return 1 # Neutro

In [11]:
# Se crea la columna sentiment
Df_reviews['sentiment'] = Df_reviews['review'].apply(sentimiento)
Df_reviews.head()

,user_id,posted,item_id,recommend,review,year,sentiment
0,76561197970982479,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,2011,2
1,76561197970982479,2011-07-15,22200,True,It's unique and worth a playthrough.,2011,2
2,76561197970982479,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...,2011,2
3,js41637,2014-06-24,251610,True,I know what you think when you see this title ...,2014,2
4,js41637,2013-09-08,227300,True,For a simple (it's actually not all that simpl...,2013,0


In [12]:
# Se guardan los datos limpios en un CSV para su posterior manipulación
Df_reviews.to_csv('australian_user_reviews_new.csv', index=False)

In [13]:
Df_reviews.info()
Df_reviews.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 48498 entries, 0 to 59304
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   user_id    48498 non-null  object        
 1   posted     48498 non-null  datetime64[ns]
 2   item_id    48498 non-null  object        
 3   recommend  48498 non-null  object        
 4   review     48498 non-null  object        
 5   year       48498 non-null  Int64         
 6   sentiment  48498 non-null  int64         
dtypes: Int64(1), datetime64[ns](1), int64(1), object(4)
memory usage: 3.0+ MB


user_id      0
posted       0
item_id      0
recommend    0
review       0
year         0
sentiment    0
dtype: int64